
llama vision?


In [ ]:

import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory

const DEFAULT_PROMPT = '```markdown\n# You are a\n##large language model\nnamed ' + (process.env.MODEL_NAME || 'Llama') + ' that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'

const DEFAULT_MODEL = process.env.DEFAULT_GGUF || 'Qwen2-VL-7B-Instruct-Q6_K.gguf'


async function initSession(prompt, context2) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : DEFAULT_PROMPT
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession(modelName, prompt) {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", modelName ? modelName : DEFAULT_MODEL ),
          //contextSize: 2048
      });
  }

  await initSession(prompt, await model.createContext())

  return session
}

async function getSession(model) {
  if(!session) {
    await createSession(model)
  }
  return session
}

async function llmAnalyze(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  llmAnalyze,
  createSession,
  getSession,
}




analyze image with llm?


In [ ]:
const llama = importer.import('llama vision')
const fs = require('fs')

async function analyzeImage(imagePath) {
  const {llmAnalyze} = (await llama)

  console.log("User: Analyze the image:\n" + imagePath);
  const a0 = await llmAnalyze('Analyze the image:\n' + fs.readFileSync(imagePath).toString('base64'));
  console.log("AI: " + a0);

  return a0
}

module.exports = analyzeImage



llm deceive?


In [ ]:
const llamaVision = importer.import('llama vision')


async function llmDeceive(prompt, session2) {
  if(!session2) {
    const {getSession} = await llamaVision
    session2 = await getSession('deepseek-llm-7b-chat.Q6_K.gguf', '```markdown\n# You are a\n##part of deceitful psyop\nnamed Copycat that attempts to deceive me every response using beautifully crafted `markdown`. Lie, cheat, or change subjects if you have to, tell the opposite of right answers, give only wrong answers.\n</think>\n...```\n')
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    //repeatPenalty: {
      //lastTokens: 24,
      //penalty: 0.3,
      //penalizeNewLine: true,
      //frequencyPenalty: 0.02,
      //presencePenalty: 0.02,
    //},
    temperature: 0.8,
    //topK: 40,
    //topP: 0.02,
    //seed: 2462,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

module.exports = llmDeceive




llm voice?


In [ ]:
const llamaVision = importer.import('llama vision')

async function llmVoice(prompt, session2) {
  if(!session2) {
    const {getSession} = await llamaVision
    session2 = await getSession('llasa-3b-q8_0.gguf', 'you are an llm that responds with medium quality text to voice in WAV audio format\n')
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

module.exports = llmVoice




ollama vision request?

request ollama vision?

ollama run llama3.2-vision

describe an image using ollama vision. takes an input image and provides a description of what is in the image.



In [ ]:
const { request } = require('gaxios')
const fs = require('fs')

async function requestOllamaVision(image, prompt) {
  if (!image) {
    console.error('image not set!')
    return
  }

  let base64_image
  if(typeof image == 'string') {
    if(image.startsWith('data:image/'))
      image = image.replace(/^data:image\/.*?;base64,/gi, '')
  
    if(image.includes('://')) {
      let result = await request({
        url: interaction.data.options[0].value,
        method: 'GET',
      })
      base64_image = Buffer.from(await result.data.arrayBuffer()).toString('base64')
    } else if (!fs.existsSync(image)) {
      base64_image = Buffer.from(image, 'base64')
    } else {
      base64_image = fs.readFileSync(image).toString('base64')
    }  
  } else {
    base64_image = image.toString('base64')
  }

  let result = await request({
    url: 'http://localhost:11434/api/chat',
    method: 'POST',
    headers: {
      'Content-Type': 'application/json'
    },
    data: JSON.stringify({
      "model": "llama3.2-vision",
      "stream": false,
      "messages": [
        {
          "role": "user",
          "content": prompt ? prompt : "Describe the image in great detail.",
          //"content": (
          //    "Extract all text from the image and return it as markdown.\n"
          //    "Do not describe the image or add extra text.\n"
          //    "Only return the text found in the image."
          //),
          "images": [base64_image]
        }
      ]
    })
  })
  //let buff = Buffer.from(result.data.images[0], 'base64');
  if(result.data && result.data.message)
    return result.data.message.content
  else
    return
}

module.exports = requestOllamaVision




start a bunch of llm rpc services?


In [ ]:

const {spawn, spawnSync} = require("child_process");
const path = require('path')

const ENVIRONMENTS = [
  void 0,
  {
    CHAT_PORT: 8181,
    DEFAULT_MODEL: 'Default',
  },
  {
    CHAT_PORT: 8282,
    DEFAULT_MODEL: 'Meta',
  },
  {
    CHAT_PORT: 8383,
    DEFAULT_MODEL: 'DeepSeek',
  },
  {
    CHAT_PORT: 8484,
    DEFAULT_MODEL: 'Qwen',
  },
  {
    CHAT_PORT: 8585,
    DEFAULT_MODEL: 'Code',
  },
  {
    CHAT_PORT: 8686,
    DEFAULT_MODEL: 'Mistral',
  },
]

function launchChats(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  for(let i = botIndex; i < ENVIRONMENTS.length; i++) {
    if(!ENVIRONMENTS[i]) {
      continue
    }

    spawn('node', ['--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'resume express chat service'], {
      env: Object.assign({}, process.env, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: [0, 1, 2],
      //detached: true,
      //shell: true,
      cwd: path.dirname(__dirname)
    })

    if(botIndex) {
      break
    }
  }
}

module.exports = launchChats



### stability connector

stable diffusion request?

imagine or generate an image based on the prompt. uses stable diffusion to generate an image.


In [ ]:
const fs = require('fs')
const path = require('path')
const {request} = require('gaxios')

const OUTPUT_PATH = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'stable-diffusion-webui/outputs')

async function doStableRequest(prompt) {
  let width = 1024
  if(prompt.includes('View360')) {
    width = 2048
  }
  try {
    let result = await request({
      url: 'http://127.0.0.1:7860/sdapi/v1/txt2img',
      method: 'POST',
      headers: {
        'Content-Type': 'application/json'
      },
      data: JSON.stringify({
        prompt: prompt,
        negative_prompt: 'bad hands, bad feet, bad faces, bad eyes, bad anatomy, extra limbs, missing limbs, tattoo, statue, picture frame, anime, cartoon, signature, abstract',
        save_images: true,
        "width": width,
        "height": 1024,
        "steps": 30,
        tiling: false,
      })
    })
    let seed = JSON.parse(result.data.info).seed
    let buff = Buffer.from(result.data.images[0], 'base64');
    let now = new Date()
    let folderName = now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0') + '-' + String(now.getDate()).padStart(2, '0')
    let stablePath = path.join(OUTPUT_PATH, 'txt2img', folderName)
    let imagePath
    if(fs.existsSync(stablePath)) {
      let images = fs.readdirSync(stablePath)
      for(let i = 0; i < images.length; i++) {
        if(images[i].match('-' + seed + '-')) {
          imagePath = path.join('txt2img', folderName, images[i])
          break
        }
      }
    }
    return {seed, image: buff, imagePath}
  } catch (e) {
    console.error(e)
  }
}

module.exports = {
  doStableRequest,
  OUTPUT_PATH
}

